In [ ]:
pip install beautifulsoup4

In [14]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time

Escrevendo SCRAPER (Prever erros)

In [2]:
def getTitle(url):
    try:
        html= urlopen(url)
    except HTTPError as e:
        return None
    try:
        bs4= BeautifulSoup(html, 'html.parser')
        title= bs4.html.h1
    except AttributeError as e:
        return None
    return title

title= getTitle('https://github.com/search?o=desc&q=stars%3A%3E100000+language%3APython&s=stars&type=Repositories&l=Python')
if title == None:
    print('Titulo não foi encontrado')
else:
    print(title)



<h1 class="sr-only" id="search-suggestions-dialog-header">Search code, repositories, users, issues, pull requests...</h1>


In [34]:
# Configurar o driver do Selenium (certifique-se de ter o driver correspondente instalado)
driver = webdriver.Chrome()

url = 'https://github.com/search?o=desc&q=stars%3A%3E100000+language%3APython&s=stars&type=Repositories&l=Python'
driver.get(url)

# Aguarde alguns segundos para garantir que a página carregue completamente (pode ser ajustado conforme necessário)
time.sleep(5)

# Obtenha o HTML da página
html = driver.page_source
bs4 = BeautifulSoup(html, 'html.parser')

# Encontrar a tag div com a classe específica que contém os resultados
results_divs = bs4.find_all('div', {'class': 'Box-sc-g0xbh4-0 cFPoqW'})

# Criar listas para armazenar os resultados
titles = []
languages = []
stars = []

# Iterar sobre os resultados e extrair as informações
for result_div in results_divs:
    title = result_div.find('div', {'class': 'Box-sc-g0xbh4-0 bBwPjs search-title'}).text.strip()
    language = result_div.find('span', {'class': 'Text-sc-17v1xeu-0 gPDEWA'})
    stars_element = result_div.find('svg', {'class': 'Link__StyledLink-sc-14289xe-0 bzaAZD'})

    # Certificar-se de que todos os elementos foram encontrados antes de tentar acessar o texto
    if language:
        language = language.text.strip()
    else:
        language = 'Not specified'

    if stars_element:
        stars_text = stars_element.text.strip()
    else:
        stars_text = '0'

    titles.append(title)
    languages.append(language)
    stars.append(stars_text)

# Criar um DataFrame do pandas a partir dos resultados
df = pd.DataFrame({'Title': titles, 'Language': languages, 'Stars': stars})

# Imprimir o DataFrame
print(df)

# Fechar o driver do Selenium
driver.quit()

                                  Title Language Stars
0               public-apis/public-apis   Python     0
1      donnemartin/system-design-primer   Python     0
2                  vinta/awesome-python   Python     0
3                  TheAlgorithms/Python   Python     0
4             jackfrued/Python-100-Days   Python     0
5                   ytdl-org/youtube-dl   Python     0
6  AUTOMATIC1111/stable-diffusion-webui   Python     0
7              huggingface/transformers   Python     0


In [97]:
import requests
from bs4 import BeautifulSoup
import csv

url = 'https://github.com/trending'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar os elementos de contêiner para os resultados
    project_containers = soup.find_all('article', {'class': 'Box-row'})

    # Criar lista para armazenar os dados
    data = []

    # Iterar sobre os resultados e extrair as informações
    for container in project_containers:
        # Procurar pelo título do projeto
        title_element = container.find('h2', {'class': 'h3'})
        title = title_element.text.strip() if title_element else 'Not specified'

        # Remover espaços e quebras de linha extras no título
        title = ' '.join(title.split())

        # Procurar pela linguagem do projeto
        language_element = container.find('span', {'itemprop': 'programmingLanguage'})
        language = language_element.text.strip() if language_element else 'x'

        # Procurar pelo número de estrelas do projeto
        stars_element = container.find('a', {'class': 'Link--muted'})
        stars = stars_element.text.strip() if stars_element else '0'

        # Procurar pelo número de estrelas hoje e forks do projeto
        meta_elements = container.find_all('a', {'class': 'Link--muted'})
        stars_today = meta_elements[1].text.strip() if len(meta_elements) > 1 else '0'
        forks = meta_elements[2].text.strip() if len(meta_elements) > 2 else '0'

        # Adicionar os dados à lista
        data.append([title, language, stars, stars_today, forks])

    # Escrever os dados no arquivo CSV
    with open('github.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=';')
        csv_writer.writerow(['ranking', 'project', 'language', 'stars', 'stars_today', 'forks'])
        
        for index, row in enumerate(data, start=1):
            csv_writer.writerow([index] + row)

    print("Dados extraídos com sucesso e gravados em 'github.csv'.")
else:
    print(f"Falha na requisição com status code: {response.status_code}")


Dados extraídos com sucesso e gravados em 'github.csv'.


In [99]:
# Ler o arquivo CSV substituindo vírgulas por pontos e ponto e vírgula por vírgula
df = pd.read_csv('github.csv', delimiter=';', decimal=',')

# Dividir a coluna "projeto" em duas colunas: "usuário" e "projeto"
df[['usuário', 'projeto']] = df['project'].str.split('/', n=1, expand=True)

# Substituir o ponto e vírgula por vírgula e a vírgula por ponto nas colunas numéricas
df[['stars', 'stars_today']] = df[['stars', 'stars_today']].replace({',': '.', ';': ','}, regex=True)

# Reorganizar as colunas
df = df[['ranking', 'usuário', 'projeto', 'language', 'stars', 'stars_today', 'forks']]

# Salvar o DataFrame de volta como um novo arquivo CSV
df.to_csv('github.csv', index=False, sep=',', decimal='.')
print("Arquivo editado salvo como 'github.csv'.")


Arquivo editado salvo como 'github.csv'.


In [104]:
df= pd.read_csv('github.csv')
display(df)

,ranking,usuário,projeto,language,stars,stars_today,forks
0,1,cloudflare,pingora,Rust,12.511,652.000,0
1,2,Lissy93,web-check,TypeScript,14.416,1.058,0
2,3,polyfillpolyfill,polyfill-service,x,6.574,634.000,0
3,4,HumanAIGC,EMO,x,4.858,390.000,0
4,5,cloudcommunity,Free-Certifications,x,19.198,1.478,0
5,6,yuzu-emu,yuzu,C++,38.541,3.990,0
6,7,microsoft,generative-ai-for-beginners,Jupyter Notebook,25.402,15.987,0
7,8,HumanAIGC,AnimateAnyone,x,13.452,845.000,0
8,9,kyegomez,BitNet,Python,838.000,70.000,0
9,10,dockur,windows,Shell,3.917,261.000,0
